In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { instance } from "@viz-js/viz";
import { RecursiveSet } from "recursive-set";

import { Char, DFA, key } from "./01-NFA-2-DFA";
import { RegExp } from "./03-RegExp-2-NFA";
import { parse } from "./RegExp-Parser";
import {
    regExpEquiv,
    regexp2DFA,
    fsm_complement,
    ProductDFA,
    StatePair,
} from "./09-Equivalence";
import { minimize } from "./07-Minimize";
import { dfa2dot } from "./FSM-2-Dot";

const viz = await instance();

# Test the Equivalence of Regular Expressions

In this notebook, we test the equivalence algorithm implemented in `09-Equivalence.ipynb`.

## 1. Standard Equivalence Testing

We start by defining a simple wrapper function `test`. This function verifies if two regular expressions $r_1$ and $r_2$ describe the same language ($L(r_1) = L(r_2)$).

It uses the `regExpEquiv` function, which performs the full $L(D_1) \setminus L(D_2) = \emptyset$ check.

In [ ]:
function test(Sigma: RecursiveSet<Char>, s1: string, s2: string): void {
    try {
        const r1: RegExp = parse(s1);
        const r2: RegExp = parse(s2);

        if (regExpEquiv(r1, r2, Sigma)) {
            console.log(`✅ Equivalent:  ${s1}  ===  ${s2}`);
        } else {
            console.log(`❌ Different:   ${s1}  !==  ${s2}`);
        }
    } catch (e) {
        console.error(`Error testing equivalence for ${s1} and ${s2}:`, e);
    }
}

We define the alphabet $\Sigma = \{a, b, c\}$ and run several test cases. These examples range from simple identities to complex structural variations.

In [ ]:
const Sigma = new RecursiveSet<Char>("a", "b", "c");

In [ ]:
test(Sigma, '(ε+a)(a+ε)*(a+ε)', 'a*');

In [ ]:
test(Sigma, "(ba)(ba)*", "b(ab)*a");

In [ ]:
test(Sigma, "(a+b+c)*(ac*b+bc*a)(a+b+c)*", "c*(a(a+c)*b+b(b+c)*a)(a+b+c)*");

In [ ]:
test(
    Sigma,
    "((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)",
    "c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*",
);

In [ ]:
test(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*");

## 2. Advanced Visual Testing

While a simple boolean result is sufficient for correctness, it doesn't explain *why* two expressions differ.
We implement an advanced **Visual Testing Framework** that provides deeper insights:

1.  **Minimizes** the resulting automata. Recall the **Myhill-Nerode Theorem**: Minimal DFAs are unique up to isomorphism. If $r_1 \equiv r_2$, their minimal DFAs must look structurally identical.
2.  **Visualizes** them side-by-side for direct comparison.
3.  **Generates Witnesses:** If they are not equivalent, we search for the shortest word $w$ that is accepted by one but not the other.

### Finding Witnesses (Counter-Examples)

To find a witness, we perform a **Breadth-First Search (BFS)** on the **Product Automaton** (specifically the difference automaton constructed by `fsm_complement`).
We start at the initial state pair $(q_{01}, q_{02})$. The queue stores states alongside the word used to reach them. The search proceeds level by level (by word length). The first accepting state we encounter corresponds to the **shortest** witness word $w$ in the symmetric difference language.

In [ ]:
function findWitness(F: ProductDFA): string | null {
    const queue: { state: StatePair; word: string }[] = [
        { state: F.q0, word: "" },
    ];
    const visited = new RecursiveSet<StatePair>(F.q0);

    let head = 0;
    while (head < queue.length) {
        const { state, word } = queue[head++];

        if (F.A.has(state)) {
            return word === "" ? "ε" : word;
        }

        for (const c of F.Sigma) {
            const nextState = F.delta.get(key(state, c));

            if (nextState && !visited.has(nextState)) {
                visited.add(nextState);
                queue.push({ state: nextState, word: word + c });
            }
        }
    }
    return null;
}

### Visual Comparison Function

The `testVisual` function orchestrates the entire comparison pipeline:

1.  **Parse & Convert:** Both regular expressions are parsed and converted to DFAs.
2.  **Minimization:** Both DFAs are minimized using the algorithm from `07-Minimize`.
    * *Implementation Note:* We cast the result to `DFA` (`as unknown as DFA`) because the visualization tool `dfa2dot` expects a standard structure (sets of primitives). Our minimized DFA consists of sets of sets (`MinState`), but since `RecursiveSet` handles nested stringification automatically, this works perfectly for visualization without extra conversion logic.
3.  **Difference Check:** We compute $L(r_1) \setminus L(r_2)$ and vice versa to find witnesses.
4.  **Rendering:** We generate SVG images for both minimal DFAs and display them side-by-side using an HTML layout.

In [ ]:
async function testVisual(
    Sigma: RecursiveSet<Char>,
    s1: string,
    s2: string,
): Promise<void> {
    console.log(`\n🔎 Inspecting: "${s1}" vs "${s2}"`);

    try {
        const r1 = parse(s1);
        const r2 = parse(s2);

        const dfa1 = regexp2DFA(r1, Sigma);
        const dfa2 = regexp2DFA(r2, Sigma);

        const min1 = minimize(dfa1) as unknown as DFA;
        const min2 = minimize(dfa2) as unknown as DFA;

        const diff1 = fsm_complement(min1, min2);
        const diff2 = fsm_complement(min2, min1);

        const witness1 = findWitness(diff1);
        const witness2 = findWitness(diff2);
        const equivalent = witness1 === null && witness2 === null;

        const dot1 = dfa2dot(min1).dot;
        const dot2 = dfa2dot(min2).dot;

        const svg1 = viz.renderString(dot1, { format: "svg" });
        const svg2 = viz.renderString(dot2, { format: "svg" });

        if (equivalent) {
            console.log("✅ RESULT: Equivalent!");
            console.log("Both expressions produce isomorphic minimal DFAs.");
        } else {
            console.log("❌ RESULT: NOT Equivalent.");
            if (witness1)
                console.log(
                    `   Witness 1: "${witness1}" is in L(r1) but NOT in L(r2).`,
                );
            if (witness2)
                console.log(
                    `   Witness 2: "${witness2}" is in L(r2) but NOT in L(r1).`,
                );
        }

        display.html(`
    <div style="display: flex; gap: 20px; border: 1px solid #ccc; padding: 10px; align-items: flex-start; background: #f9f9f9;">
        <div style="flex: 1; text-align: center;">
            <h4 style="margin: 5px;">RegExp 1 (Minimized)</h4>
            <code style="background: #eee; padding: 2px 5px; border-radius: 3px;">${s1}</code>
            <div style="margin-top: 10px;">${svg1}</div>
        </div>
        <div style="border-left: 2px solid #ccc; align-self: stretch;"></div>
        <div style="flex: 1; text-align: center;">
            <h4 style="margin: 5px;">RegExp 2 (Minimized)</h4>
            <code style="background: #eee; padding: 2px 5px; border-radius: 3px;">${s2}</code>
            <div style="margin-top: 10px;">${svg2}</div>
        </div>
    </div>
    `);
    } catch (e) {
        console.error(`Error:`, e);
    }
}


In [ ]:
console.log("--- VISUAL RE-RUN ---");

await testVisual(Sigma, "(ε+a)(a+ε)*(a+ε)", "a*");

await testVisual(Sigma, "(ba)(ba)*", "b(ab)*a");

await testVisual(
    Sigma,
    "(a+b+c)*(ac*b+bc*a)(a+b+c)*",
    "c*(a(a+c)*b+b(b+c)*a)(a+b+c)*",
);

await testVisual(
    Sigma,
    "((c*ac*)*(c*ac*)*(c*bc*)(c*bc*)*)+((c*bc*)(c*bc*)*(c*ac*)(c*ac*)*)",
    "c*(a*(a+c)*b+b*(b+c)*a)(a+b+c)*",
);

await testVisual(Sigma, "(a+b)*a(a+b)*a(a+b)*a(a+b)*", "a*b*ab*ab*ab*a*");